In [1]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.5 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=3cb24ca538194e6de28e04e893a777cde3cf393d89921f74f99e8d0baf94f50d
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, rand, isnan, when, count, expr, isnull
# from pyspark.sql.functions import col, count, isnan, when
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.sql import functions as F

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better
spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/25 19:39:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
df18 = spark.read.csv("/kaggle/input/flight-delay-dataset-20182022/Combined_Flights_2018.csv", header=True, inferSchema=True)

In [6]:
df19 = spark.read.csv("/kaggle/input/flight-delay-dataset-20182022/Combined_Flights_2019.csv", header=True, inferSchema=True)

In [7]:
df20 = spark.read.csv("/kaggle/input/flight-delay-dataset-20182022/Combined_Flights_2020.csv", header=True, inferSchema=True)

In [8]:
df21 = spark.read.csv("/kaggle/input/flight-delay-dataset-20182022/Combined_Flights_2021.csv", header=True, inferSchema=True)

In [9]:
df22 = spark.read.csv("/kaggle/input/flight-delay-dataset-20182022/Combined_Flights_2022.csv", header=True, inferSchema=True)

In [10]:
df18_true_cancelled = df18.filter(col('Cancelled') == True)
df19_true_cancelled = df19.filter(col('Cancelled') == True)
df20_true_cancelled = df20.filter(col('Cancelled') == True)
df21_true_cancelled = df21.filter(col('Cancelled') == True)
df22_true_cancelled = df22.filter(col('Cancelled') == True)

In [11]:
true_cancelled_df = df18_true_cancelled.union(df19_true_cancelled).union(df20_true_cancelled).union(df21_true_cancelled).union(df22_true_cancelled)
true_cancelled_df

23/12/25 19:42:46 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


FlightDate,Airline,Origin,Dest,Cancelled,Diverted,CRSDepTime,DepTime,DepDelayMinutes,DepDelay,ArrTime,ArrDelayMinutes,AirTime,CRSElapsedTime,ActualElapsedTime,Distance,Year,Quarter,Month,DayofMonth,DayOfWeek,Marketing_Airline_Network,Operated_or_Branded_Code_Share_Partners,DOT_ID_Marketing_Airline,IATA_Code_Marketing_Airline,Flight_Number_Marketing_Airline,Operating_Airline,DOT_ID_Operating_Airline,IATA_Code_Operating_Airline,Tail_Number,Flight_Number_Operating_Airline,OriginAirportID,OriginAirportSeqID,OriginCityMarketID,OriginCityName,OriginState,OriginStateFips,OriginStateName,OriginWac,DestAirportID,DestAirportSeqID,DestCityMarketID,DestCityName,DestState,DestStateFips,DestStateName,DestWac,DepDel15,DepartureDelayGroups,DepTimeBlk,TaxiOut,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrDelay,ArrDel15,ArrivalDelayGroups,ArrTimeBlk,DistanceGroup,DivAirportLandings
2018-01-17,Endeavor Air Inc.,ATL,ABY,true,false,1037,NULL,NULL,NULL,NULL,NULL,NULL,60.0,NULL,145.0,2018,1,1,17,3,DL,DL_CODESHARE,19790,DL,3298,9E,20363,9E,N8972E,3298,10397,1039707,30397,"Atlanta, GA",GA,13,Georgia,34,10146,1014602,30146,"Albany, GA",GA,13,Georgia,34,NULL,NULL,1000-1059,NULL,NULL,NULL,NULL,1137,NULL,NULL,NULL,1100-1159,1,0.0
2018-01-12,Endeavor Air Inc.,ATL,EVV,true,false,939,NULL,NULL,NULL,NULL,NULL,NULL,89.0,NULL,350.0,2018,1,1,12,5,DL,DL_CODESHARE,19790,DL,3299,9E,20363,9E,N8928A,3299,10397,1039707,30397,"Atlanta, GA",GA,13,Georgia,34,11612,1161206,31612,"Evansville, IN",IN,18,Indiana,42,NULL,NULL,0900-0959,NULL,NULL,NULL,NULL,1008,NULL,NULL,NULL,1000-1059,2,0.0
2018-01-17,Endeavor Air Inc.,ATL,EVV,true,false,939,NULL,NULL,NULL,NULL,NULL,NULL,89.0,NULL,350.0,2018,1,1,17,3,DL,DL_CODESHARE,19790,DL,3299,9E,20363,9E,N8688C,3299,10397,1039707,30397,"Atlanta, GA",GA,13,Georgia,34,11612,1161206,31612,"Evansville, IN",IN,18,Indiana,42,NULL,NULL,0900-0959,NULL,NULL,NULL,NULL,1008,NULL,NULL,NULL,1000-1059,2,0.0
2018-01-04,Endeavor Air Inc.,ATL,EWN,true,false,1225,NULL,NULL,NULL,NULL,NULL,NULL,85.0,NULL,433.0,2018,1,1,4,4,DL,DL_CODESHARE,19790,DL,3301,9E,20363,9E,N836AY,3301,10397,1039707,30397,"Atlanta, GA",GA,13,Georgia,34,11617,1161708,31617,New Bern/Morehead...,NC,37,North Carolina,36,NULL,NULL,1200-1259,NULL,NULL,NULL,NULL,1350,NULL,NULL,NULL,1300-1359,2,0.0
2018-01-04,Endeavor Air Inc.,EWN,ATL,true,false,1415,NULL,NULL,NULL,NULL,NULL,NULL,110.0,NULL,433.0,2018,1,1,4,4,DL,DL_CODESHARE,19790,DL,3301,9E,20363,9E,N836AY,3301,11617,1161708,31617,New Bern/Morehead...,NC,37,North Carolina,36,10397,1039707,30397,"Atlanta, GA",GA,13,Georgia,34,NULL,NULL,1400-1459,NULL,NULL,NULL,NULL,1605,NULL,NULL,NULL,1600-1659,2,0.0
2018-01-17,Endeavor Air Inc.,ATL,FAY,true,false,1708,NULL,NULL,NULL,NULL,NULL,NULL,80.0,NULL,331.0,2018,1,1,17,3,DL,DL_CODESHARE,19790,DL,3302,9E,20363,9E,N980EV,3302,10397,1039707,30397,"Atlanta, GA",GA,13,Georgia,34,11641,1164102,31641,"Fayetteville, NC",NC,37,North Carolina,36,NULL,NULL,1700-1759,NULL,NULL,NULL,NULL,1828,NULL,NULL,NULL,1800-1859,2,0.0
2018-01-16,Endeavor Air Inc.,FAY,ATL,true,false,1853,NULL,NULL,NULL,NULL,NULL,NULL,97.0,NULL,331.0,2018,1,1,16,2,DL,DL_CODESHARE,19790,DL,3302,9E,20363,9E,N8683B,3302,11641,1164102,31641,"Fayetteville, NC",NC,37,North Carolina,36,10397,1039707,30397,"Atlanta, GA",GA,13,Georgia,34,NULL,NULL,1800-1859,NULL,NULL,NULL,NULL,2030,NULL,NULL,NULL,2000-2059,2,0.0
2018-01-17,Endeavor Air Inc.,FAY,ATL,true,false,1853,NULL,NULL,NULL,NULL,NULL,NULL,97.0,NULL,331.0,2018,1,1,17,3,DL,DL_CODESHARE,19790,DL,3302,9E,20363,9E,N980EV,3302,11641,1164102,31641,"Fayetteville, NC",NC,37,North Carolina,36,10397,1039707,30397,"Atlanta, GA",GA,13,Georgia,34,NULL,NULL,1800-1859,NULL,NULL,NULL,NULL,2030,NULL,NULL,NULL,2000-2059,2,0.0
2018-01-04,Endeavor Air Inc.,RDU,BOS,true,false,1420,NULL,NULL,NULL,NULL,NULL,NULL,119.0,NULL,612.0,2018,1,1,4,4,DL,DL_CODESHARE,19790,DL,3303,9E,20363,9E,N905XJ,3303,14492,1449202,34492,"Raleigh/Durham, NC",NC,37,North Carolina,36,10721,1072102,30721,"Boston, MA",MA,25,Massachusetts,13,NULL,NULL,1400-

In [12]:
num_rows = true_cancelled_df.count()
num_columns = len(true_cancelled_df.columns)

print("Number of Rows: {}".format(num_rows))
print("Number of Columns: {}".format(num_columns))

Number of Rows: 777267
Number of Columns: 61


In [13]:
df18_false_cancelled = df18.filter(col('Cancelled') == False)
df19_false_cancelled = df19.filter(col('Cancelled') == False)
df20_false_cancelled = df20.filter(col('Cancelled') == False)
df21_false_cancelled = df21.filter(col('Cancelled') == False)
df22_false_cancelled = df22.filter(col('Cancelled') == False)

In [14]:
false_cancelled_df = df18_false_cancelled.union(df19_false_cancelled).union(df20_false_cancelled).union(df21_false_cancelled).union(df22_false_cancelled)
false_cancelled_df

FlightDate,Airline,Origin,Dest,Cancelled,Diverted,CRSDepTime,DepTime,DepDelayMinutes,DepDelay,ArrTime,ArrDelayMinutes,AirTime,CRSElapsedTime,ActualElapsedTime,Distance,Year,Quarter,Month,DayofMonth,DayOfWeek,Marketing_Airline_Network,Operated_or_Branded_Code_Share_Partners,DOT_ID_Marketing_Airline,IATA_Code_Marketing_Airline,Flight_Number_Marketing_Airline,Operating_Airline,DOT_ID_Operating_Airline,IATA_Code_Operating_Airline,Tail_Number,Flight_Number_Operating_Airline,OriginAirportID,OriginAirportSeqID,OriginCityMarketID,OriginCityName,OriginState,OriginStateFips,OriginStateName,OriginWac,DestAirportID,DestAirportSeqID,DestCityMarketID,DestCityName,DestState,DestStateFips,DestStateName,DestWac,DepDel15,DepartureDelayGroups,DepTimeBlk,TaxiOut,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrDelay,ArrDel15,ArrivalDelayGroups,ArrTimeBlk,DistanceGroup,DivAirportLandings
2018-01-23,Endeavor Air Inc.,ABY,ATL,false,false,1202,1157.0,0.0,-5.0,1256.0,0.0,38.0,62.0,59.0,145.0,2018,1,1,23,2,DL,DL_CODESHARE,19790,DL,3298,9E,20363,9E,N8928A,3298,10146,1014602,30146,"Albany, GA",GA,13,Georgia,34,10397,1039707,30397,"Atlanta, GA",GA,13,Georgia,34,0.0,-1.0,1200-1259,14.0,1211.0,1249.0,7.0,1304,-8.0,0.0,-1.0,1300-1359,1,0.0
2018-01-24,Endeavor Air Inc.,ABY,ATL,false,false,1202,1157.0,0.0,-5.0,1258.0,0.0,36.0,62.0,61.0,145.0,2018,1,1,24,3,DL,DL_CODESHARE,19790,DL,3298,9E,20363,9E,N800AY,3298,10146,1014602,30146,"Albany, GA",GA,13,Georgia,34,10397,1039707,30397,"Atlanta, GA",GA,13,Georgia,34,0.0,-1.0,1200-1259,13.0,1210.0,1246.0,12.0,1304,-6.0,0.0,-1.0,1300-1359,1,0.0
2018-01-25,Endeavor Air Inc.,ABY,ATL,false,false,1202,1153.0,0.0,-9.0,1302.0,0.0,40.0,62.0,69.0,145.0,2018,1,1,25,4,DL,DL_CODESHARE,19790,DL,3298,9E,20363,9E,N8836A,3298,10146,1014602,30146,"Albany, GA",GA,13,Georgia,34,10397,1039707,30397,"Atlanta, GA",GA,13,Georgia,34,0.0,-1.0,1200-1259,18.0,1211.0,1251.0,11.0,1304,-2.0,0.0,-1.0,1300-1359,1,0.0
2018-01-26,Endeavor Air Inc.,ABY,ATL,false,false,1202,1150.0,0.0,-12.0,1253.0,0.0,35.0,62.0,63.0,145.0,2018,1,1,26,5,DL,DL_CODESHARE,19790,DL,3298,9E,20363,9E,N800AY,3298,10146,1014602,30146,"Albany, GA",GA,13,Georgia,34,10397,1039707,30397,"Atlanta, GA",GA,13,Georgia,34,0.0,-1.0,1200-1259,17.0,1207.0,1242.0,11.0,1304,-11.0,0.0,-1.0,1300-1359,1,0.0
2018-01-27,Endeavor Air Inc.,ABY,ATL,false,false,1400,1355.0,0.0,-5.0,1459.0,0.0,36.0,60.0,64.0,145.0,2018,1,1,27,6,DL,DL_CODESHARE,19790,DL,3298,9E,20363,9E,N8839E,3298,10146,1014602,30146,"Albany, GA",GA,13,Georgia,34,10397,1039707,30397,"Atlanta, GA",GA,13,Georgia,34,0.0,-1.0,1400-1459,17.0,1412.0,1448.0,11.0,1500,-1.0,0.0,-1.0,1500-1559,1,0.0
2018-01-28,Endeavor Air Inc.,ABY,ATL,false,false,1202,1202.0,NULL,NULL,1326.0,22.0,37.0,62.0,84.0,145.0,2018,1,1,28,7,DL,DL_CODESHARE,19790,DL,3298,9E,20363,9E,N8918B,3298,10146,1014602,30146,"Albany, GA",GA,13,Georgia,34,10397,1039707,30397,"Atlanta, GA",GA,13,Georgia,34,NULL,NULL,1200-1259,34.0,1236.0,1313.0,13.0,1304,22.0,1.0,1.0,1300-1359,1,0.0
2018-01-29,Endeavor Air Inc.,ABY,ATL,false,false,1202,1204.0,2.0,2.0,1303.0,0.0,34.0,62.0,59.0,145.0,2018,1,1,29,1,DL,DL_CODESHARE,19790,DL,3298,9E,20363,9E,N8783E,3298,10146,1014602,30146,"Albany, GA",GA,13,Georgia,34,10397,1039707,30397,"Atlanta, GA",GA,13,Georgia,34,0.0,0.0,1200-1259,15.0,1219.0,1253.0,10.0,1304,-1.0,0.0,-1.0,1300-1359,1,0.0
2018-01-30,Endeavor Air Inc.,ABY,ATL,false,false,1202,1153.0,0.0,-9.0,1255.0,0.0,44.0,62.0,62.0,145.0,2018,1,1,30,2,DL,DL_CODESHARE,19790,DL,3298,9E,20363,9E,N8946A,3298,10146,1014602,30146,"Albany, GA",GA,13,Georgia,34,10397,1039707,30397,"Atlanta, GA",GA,13,Georgia,34,0.0,-1.0,1200-1259,7.0,1200.0,1244.0,11.0,1304,-9.0,0.0,-1.0,1300-1359,1,0.0
2018-01-31,Endeavor Air Inc.,ABY,ATL,false,false,1202,1153.0,0.0,-9.0,1304.0,NULL,37.0,62.0,71.0,145.0,2018,1,1,31,3,DL,DL_CODESHARE,19790,DL,3298,9E,20363,9E,N8918B,3298,10146,1014602,30146,"Albany, GA",GA,13,Georgia,34,10397,1039707,30397,"Atlanta, GA",GA,13,Georgia,34,0.0,-1.0,1200-1259,26.0,1219.0,1256.0,8.0,1304,NULL,NULL,NULL,130

In [17]:
num_rows = false_cancelled_df.count()
num_columns = len(false_cancelled_df.columns)

print("Number of Rows: {}".format(num_rows))
print("Number of Columns: {}".format(num_columns))

Number of Rows: 28416515
Number of Columns: 61


In [15]:
false_cancelled_dfC = false_cancelled_df.dropna()

In [16]:
num_rows = false_cancelled_dfC.count()
num_columns = len(false_cancelled_dfC.columns)

print("Number of Rows: {}".format(num_rows))
print("Number of Columns: {}".format(num_columns))

Number of Rows: 28339510
Number of Columns: 61


In [17]:
filtered_df = df.filter(col('Cancelled') == False)

# Get unique values of the 'Diverted' column
unique_diverted_values = filtered_df.select('Diverted').distinct()

In [19]:
unique_diverted_values

Diverted
true
false


In [18]:
r = int(1.6 * 777267)
false_cancelled_sampled = false_cancelled_dfC.sample(withReplacement=False, fraction = r / false_cancelled_df.count(), seed=42)
df = true_cancelled_df.union(false_cancelled_sampled)

In [19]:
df

FlightDate,Airline,Origin,Dest,Cancelled,Diverted,CRSDepTime,DepTime,DepDelayMinutes,DepDelay,ArrTime,ArrDelayMinutes,AirTime,CRSElapsedTime,ActualElapsedTime,Distance,Year,Quarter,Month,DayofMonth,DayOfWeek,Marketing_Airline_Network,Operated_or_Branded_Code_Share_Partners,DOT_ID_Marketing_Airline,IATA_Code_Marketing_Airline,Flight_Number_Marketing_Airline,Operating_Airline,DOT_ID_Operating_Airline,IATA_Code_Operating_Airline,Tail_Number,Flight_Number_Operating_Airline,OriginAirportID,OriginAirportSeqID,OriginCityMarketID,OriginCityName,OriginState,OriginStateFips,OriginStateName,OriginWac,DestAirportID,DestAirportSeqID,DestCityMarketID,DestCityName,DestState,DestStateFips,DestStateName,DestWac,DepDel15,DepartureDelayGroups,DepTimeBlk,TaxiOut,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrDelay,ArrDel15,ArrivalDelayGroups,ArrTimeBlk,DistanceGroup,DivAirportLandings
2018-01-17,Endeavor Air Inc.,ATL,ABY,true,false,1037,NULL,NULL,NULL,NULL,NULL,NULL,60.0,NULL,145.0,2018,1,1,17,3,DL,DL_CODESHARE,19790,DL,3298,9E,20363,9E,N8972E,3298,10397,1039707,30397,"Atlanta, GA",GA,13,Georgia,34,10146,1014602,30146,"Albany, GA",GA,13,Georgia,34,NULL,NULL,1000-1059,NULL,NULL,NULL,NULL,1137,NULL,NULL,NULL,1100-1159,1,0.0
2018-01-12,Endeavor Air Inc.,ATL,EVV,true,false,939,NULL,NULL,NULL,NULL,NULL,NULL,89.0,NULL,350.0,2018,1,1,12,5,DL,DL_CODESHARE,19790,DL,3299,9E,20363,9E,N8928A,3299,10397,1039707,30397,"Atlanta, GA",GA,13,Georgia,34,11612,1161206,31612,"Evansville, IN",IN,18,Indiana,42,NULL,NULL,0900-0959,NULL,NULL,NULL,NULL,1008,NULL,NULL,NULL,1000-1059,2,0.0
2018-01-17,Endeavor Air Inc.,ATL,EVV,true,false,939,NULL,NULL,NULL,NULL,NULL,NULL,89.0,NULL,350.0,2018,1,1,17,3,DL,DL_CODESHARE,19790,DL,3299,9E,20363,9E,N8688C,3299,10397,1039707,30397,"Atlanta, GA",GA,13,Georgia,34,11612,1161206,31612,"Evansville, IN",IN,18,Indiana,42,NULL,NULL,0900-0959,NULL,NULL,NULL,NULL,1008,NULL,NULL,NULL,1000-1059,2,0.0
2018-01-04,Endeavor Air Inc.,ATL,EWN,true,false,1225,NULL,NULL,NULL,NULL,NULL,NULL,85.0,NULL,433.0,2018,1,1,4,4,DL,DL_CODESHARE,19790,DL,3301,9E,20363,9E,N836AY,3301,10397,1039707,30397,"Atlanta, GA",GA,13,Georgia,34,11617,1161708,31617,New Bern/Morehead...,NC,37,North Carolina,36,NULL,NULL,1200-1259,NULL,NULL,NULL,NULL,1350,NULL,NULL,NULL,1300-1359,2,0.0
2018-01-04,Endeavor Air Inc.,EWN,ATL,true,false,1415,NULL,NULL,NULL,NULL,NULL,NULL,110.0,NULL,433.0,2018,1,1,4,4,DL,DL_CODESHARE,19790,DL,3301,9E,20363,9E,N836AY,3301,11617,1161708,31617,New Bern/Morehead...,NC,37,North Carolina,36,10397,1039707,30397,"Atlanta, GA",GA,13,Georgia,34,NULL,NULL,1400-1459,NULL,NULL,NULL,NULL,1605,NULL,NULL,NULL,1600-1659,2,0.0
2018-01-17,Endeavor Air Inc.,ATL,FAY,true,false,1708,NULL,NULL,NULL,NULL,NULL,NULL,80.0,NULL,331.0,2018,1,1,17,3,DL,DL_CODESHARE,19790,DL,3302,9E,20363,9E,N980EV,3302,10397,1039707,30397,"Atlanta, GA",GA,13,Georgia,34,11641,1164102,31641,"Fayetteville, NC",NC,37,North Carolina,36,NULL,NULL,1700-1759,NULL,NULL,NULL,NULL,1828,NULL,NULL,NULL,1800-1859,2,0.0
2018-01-16,Endeavor Air Inc.,FAY,ATL,true,false,1853,NULL,NULL,NULL,NULL,NULL,NULL,97.0,NULL,331.0,2018,1,1,16,2,DL,DL_CODESHARE,19790,DL,3302,9E,20363,9E,N8683B,3302,11641,1164102,31641,"Fayetteville, NC",NC,37,North Carolina,36,10397,1039707,30397,"Atlanta, GA",GA,13,Georgia,34,NULL,NULL,1800-1859,NULL,NULL,NULL,NULL,2030,NULL,NULL,NULL,2000-2059,2,0.0
2018-01-17,Endeavor Air Inc.,FAY,ATL,true,false,1853,NULL,NULL,NULL,NULL,NULL,NULL,97.0,NULL,331.0,2018,1,1,17,3,DL,DL_CODESHARE,19790,DL,3302,9E,20363,9E,N980EV,3302,11641,1164102,31641,"Fayetteville, NC",NC,37,North Carolina,36,10397,1039707,30397,"Atlanta, GA",GA,13,Georgia,34,NULL,NULL,1800-1859,NULL,NULL,NULL,NULL,2030,NULL,NULL,NULL,2000-2059,2,0.0
2018-01-04,Endeavor Air Inc.,RDU,BOS,true,false,1420,NULL,NULL,NULL,NULL,NULL,NULL,119.0,NULL,612.0,2018,1,1,4,4,DL,DL_CODESHARE,19790,DL,3303,9E,20363,9E,N905XJ,3303,14492,1449202,34492,"Raleigh/Durham, NC",NC,37,North Carolina,36,10721,1072102,30721,"Boston, MA",MA,25,Massachusetts,13,NULL,NULL,1400-

In [20]:
num_rows = df.count()
num_columns = len(df.columns)

print("Number of Rows: {}".format(num_rows))
print("Number of Columns: {}".format(num_columns))

Number of Rows: 2017118
Number of Columns: 61


In [23]:
df = df.withColumn("random", rand()).orderBy("random").drop("random")

In [21]:
df.printSchema()

root
 |-- FlightDate: date (nullable = true)
 |-- Airline: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Cancelled: boolean (nullable = true)
 |-- Diverted: boolean (nullable = true)
 |-- CRSDepTime: integer (nullable = true)
 |-- DepTime: double (nullable = true)
 |-- DepDelayMinutes: double (nullable = true)
 |-- DepDelay: double (nullable = true)
 |-- ArrTime: double (nullable = true)
 |-- ArrDelayMinutes: double (nullable = true)
 |-- AirTime: double (nullable = true)
 |-- CRSElapsedTime: double (nullable = true)
 |-- ActualElapsedTime: double (nullable = true)
 |-- Distance: double (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Quarter: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- Marketing_Airline_Network: string (nullable = true)
 |-- Operated_or_Branded_Code_Share_Partners: string (nullable = true)
 |

In [22]:
df.describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
Airline,2017118,None,None,Air Wisconsin Airlines Corp,Virgin America
Origin,2017118,None,None,ABE,YUM
Dest,2017118,None,None,ABE,YUM
CRSDepTime,2017118,1338.7274839647457,486.29855908807747,1,2359
DepTime,1255486,1331.2379086664448,495.6757022649889,1.0,2400.0
DepDelayMinutes,1255147,13.293450089909788,47.496296785349564,0.0,2501.0
DepDelay,1255147,9.828355563133243,48.604884680756555,-118.0,2501.0
ArrTime,1239860,1467.695368831965,525.6639129804317,1.0,2400.0
ArrDelayMinutes,1239851,12.79955575307033,45.69152954761018,0.0,2486.0


In [24]:
df.coalesce(1).write.csv("/kaggle/working/balanced-samples-of-flight-status", header=True, mode="overwrite")